In [24]:

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:

import os
import cv2
import numpy as np

def load_utkface_dataset(data_dir, image_size=(100, 100), max_images=1000):
    images = []
    ages = []
    image_files = [f for f in os.listdir(data_dir) if f.endswith('.jpg')][:max_images]

    for file in image_files:
        try:
            age = int(file.split('_')[0])
            img_path = os.path.join(data_dir, file)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, image_size)
                images.append(img)
                ages.append(age)
        except Exception as e:
            print(f"Skipping {file}: {e}")

    return np.array(images), np.array(ages)

# Update this path to the actual folder location in your Google Drive
data_path = '/content/drive/MyDrive/UTKFace'  # adjust if different
X, y = load_utkface_dataset(data_path)



In [27]:

bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 100]
y_binned = np.digitize(y, bins) - 1
y_cat = to_categorical(y_binned, num_classes=len(bins))

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)


In [35]:
import os
import cv2
import numpy as np

IMG_SIZE = 128  # or 224 if preferred
DATA_DIR = "/content/drive/MyDrive/UTKFace"  # <-- change if needed

def age_to_bucket(age):
    if age <= 20:
        return 0
    elif age <= 40:
        return 1
    elif age <= 60:
        return 2
    else:
        return 3

X_data = []
y_data = []

# Loop through all image files
for filename in os.listdir(DATA_DIR):
    if filename.endswith(".jpg"):
        try:
            age = int(filename.split("_")[0])  # extract age from filename
            img_path = os.path.join(DATA_DIR, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  # resize here
            img = img / 255.0  # normalize
            X_data.append(img)
            y_data.append(age_to_bucket(age))  # bucketed label
        except Exception as e:
            print(f"Skipping {filename} due to error: {e}")
            continue

X_data = np.array(X_data)
y_data = np.array(y_data)


In [33]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(len(bins), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 134, 134,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 64, 64,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 64, 64,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 64, 64,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 66, 66,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 32, 32,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 32, 32,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 32, 32,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 32, 32,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 32, 32,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 32, 32,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 32, 32,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 32, 32,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 32, 32,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 32, 32,    │      1,024 │ conv2_block1_3_c

 Total params: 23,851,274 (90.99 MB)

 Trainable params: 263,562 (1.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [36]:
IMG_SIZE = 128  # Use this throughout your code

# When loading or preprocessing:
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

# Or if you already have X_train in (100, 100, 3):
X_train = np.array([cv2.resize(img, (IMG_SIZE, IMG_SIZE)) for img in X_train])
X_test = np.array([cv2.resize(img, (IMG_SIZE, IMG_SIZE)) for img in X_test])
history = model.fit(X_train, y_train, validation_split=0.1, epochs=10, batch_size=32,
                    callbacks=[EarlyStopping(patience=3)])



Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.2927 - loss: 2.3157 - val_accuracy: 0.9688 - val_loss: 0.3652
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.9197 - loss: 0.5266 - val_accuracy: 0.9688 - val_loss: 0.2357
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.9116 - loss: 0.4062 - val_accuracy: 0.9688 - val_loss: 0.1334
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9394 - loss: 0.1700 - val_accuracy: 0.9688 - val_loss: 0.0881
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9526 - loss: 0.1227 - val_accuracy: 1.0000 - val_loss: 0.0663
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.9799 - loss: 0.0836 - val_accuracy: 1.0000 - val_loss: 0.0357
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.9615 - loss: 0.0793 - val_accuracy: 1.0000 - val_loss: 0.0261
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9821 - loss: 0.0541 - val_accuracy: 1.0000 - val_loss: 0.0220
Epoch 9/

In [37]:

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
acc = accuracy_score(y_true_classes, y_pred_classes)
print(f"Accuracy: {acc:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step
Accuracy: 1.0000
